## 训练

In [1]:
from muzero.buffer_utils import Buffer
from muzero.config import PLANE_NUM, MuZeroConfig
from muzero.self_play import SelfPlay
from muzero.buffer_utils import make_target
import xqcpp
import numpy as np


In [2]:
init_fen = "3k5/2P1P4/9/9/9/9/9/9/4p1p2/5K3 r - 100 0 190"
# init_fen = "2r2k3/6R1C/b4N1rb/9/5n3/5C3/6n2/5p3/4p4/5K1R1 r - 110 0 180"
config = MuZeroConfig()
config.batch_size = 128
config.training_steps = 200
config.num_simulations = 60


In [3]:
buffer = Buffer(config)


In [4]:
from muzero.models import MuZeroNetwork
import torch


In [5]:
from muzero.trainer_utils import update_weights, update_lr


In [6]:
model = MuZeroNetwork(config)
model.to("cuda")
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=config.lr_init,
    momentum=config.momentum,
    weight_decay=config.weight_decay,
)


In [11]:
import random
import time


In [12]:
def selfplay():
    pass
players = [SelfPlay(config, i, init_fen) for i in range(16)]


In [13]:
batch_time = 0


In [14]:
for training_step in range(config.training_steps):
    start = time.time()
    player = random.choice(players)
    gh = player.rollout(model, training_step)
    buffer.save_game(gh)
    index_batch, batch = buffer.get_batch()
    # 训练
    # model.to("cuda")
    model.train()
    (
        total_loss,
        value_loss,
        reward_loss,
        policy_loss,
    ) = update_weights(batch, model, optimizer, config, False)
    update_lr(training_step, optimizer, config)
    if training_step % 10 == 0:
        batch_time += time.time() - start
        print(
            "{:>7d}nth loss [total:{:.2f} value:{:.2f} reward:{:.2f} policy:{:.2f}] lr:{:.5f} duration:{:.5f}".format(
                training_step,
                total_loss,
                value_loss,
                reward_loss,
                policy_loss,
                optimizer.param_groups[0]["lr"],
                batch_time,
            )
        )
        batch_time = 0


      0nth loss [total:35.24 value:0.20 reward:0.05 policy:35.14] lr:0.00200 duration:77.41420
     10nth loss [total:3.77 value:0.04 reward:0.05 policy:3.71] lr:0.00200 duration:8.67228
     20nth loss [total:2.32 value:0.04 reward:0.02 policy:2.29] lr:0.00200 duration:6.89903
     30nth loss [total:1.06 value:0.05 reward:0.01 policy:1.05] lr:0.00188 duration:6.59749
     40nth loss [total:0.60 value:0.03 reward:0.00 policy:0.59] lr:0.00176 duration:8.60835
     50nth loss [total:0.86 value:0.01 reward:0.00 policy:0.86] lr:0.00164 duration:7.01295
     60nth loss [total:0.61 value:0.00 reward:0.00 policy:0.61] lr:0.00153 duration:6.73122
     70nth loss [total:0.32 value:0.00 reward:0.00 policy:0.32] lr:0.00141 duration:5.85175
     80nth loss [total:0.50 value:0.00 reward:0.00 policy:0.50] lr:0.00129 duration:5.99368
     90nth loss [total:0.44 value:0.00 reward:0.00 policy:0.44] lr:0.00117 duration:6.20480
    100nth loss [total:0.50 value:0.00 reward:0.00 policy:0.50] lr:0.00105 du

In [15]:
# 存储模型
torch.save(model.get_weights(), "model_weights.pth")
